In [15]:
import sys 
sys.path.append("..") 

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp 

print("Numpy version: ", np.__version__)
print("Scipy version: ", sp.__version__)

from non_Hermitian_random_matrix import non_hermitian_symmetry_list, gap_types, is_complexification
from non_Hermitian_random_matrix import DeformHamiltonian, DeformHamiltonianGenerator

from machine_learning import Cluster
from topology_table import DeformTopologicalModel
from functools import partial
import itertools

from multiprocessing import Pool
from tools import save_data, save_data_M, load_data, load_data_M

Numpy version:  1.23.5
Scipy version:  1.11.1


In [16]:
# save_data(non_hermitian_symmetry_list, np.zeros(len(non_hermitian_symmetry_list)), "test.h5")
# _list, _val = load_data("test.h5")
# print(_list[1].decode('UTF-8'))
# print(_val[0])

In [17]:
gap_types

['real line', 'imaginary line', 'point']

# Run in parallel

In [18]:
n_sample = 500
n_band = 8
n_mesh = 1 # very fast mode

def sim_func_Q(Q1, Q2):
    nQ = len(Q1)
    for n in range(nQ):
        _Q = Q1[n]+Q2[n]
        for v in np.linalg.eigvalsh(_Q):
            if np.log10(np.abs(v)) < -10:
                return 0
    return 1

def get_kpoints(n_dim, n_mesh):
    '''
    Get the k points
    '''
    if n_dim == 1:
        kpoints = [[k] for k in np.linspace(0, np.pi, n_mesh)]
    else:
        values = [np.linspace(0, np.pi, n_mesh) for i in range(n_dim)]
        kpoints = list(itertools.product(*values))

    return kpoints

def fast_similarity_func(model1, model2, n_dim, n_mesh=3):
    kpoints = get_kpoints(n_dim, n_mesh=n_mesh)
    Q1 = model1.calculate_Q(kpoints)
    Q2 = model2.calculate_Q(kpoints)
    return sim_func_Q(Q1, Q2)

def worker(non_Hermitian_symmetry_class, n_dim, n_band, gap_type, n_sample, n_mesh):
    print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
        sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))

    generator  = DeformHamiltonianGenerator(
            n=n_band, n_dim=n_dim, 
            non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
            verbose=False)
    hamiltonians = generator.generate(n_sample=n_sample)
    models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
    
    similarity_func = partial(fast_similarity_func, n_dim=n_dim, n_mesh=n_mesh)
    cluster = Cluster(similarity_function=similarity_func)
    center_indices, group_number = cluster.fit(models)
    
    filename = "./data/data_{symmetry_class}_{gap_type}_dim_{n_dim}.h5".format(
        symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim)
    save_data(center_indices, group_number, filename)
    
    #return len(group_number)

n_core = len(non_hermitian_symmetry_list)

#['real line', 'imaginary line', 'point']
for gap_type in gap_types:
    for n_dim in range(7, 9+1):

        _worker = partial(worker, n_dim=n_dim, n_band=n_band, gap_type=gap_type, n_sample=n_sample, n_mesh=n_mesh)
    
        with Pool(n_core) as pool:
            pool.map(_worker, non_hermitian_symmetry_list)

Calculating A: real line dim = 7 n_band=8 n_sample=500
Calculating AIII: real line dim = 7 n_band=8 n_sample=500
Calculating BDI+: real line dim = 7 n_band=8 n_sample=500
Calculating BDI: real line dim = 7 n_band=8 n_sample=500
Calculating AI: real line dim = 7 n_band=8 n_sample=500
Calculating CII: real line dim = 7 n_band=8 n_sample=500
Calculating DIII+: real line dim = 7 n_band=8 n_sample=500
Calculating DIII: real line dim = 7 n_band=8 n_sample=500

Calculating C: real line dim = 7 n_band=8 n_sample=500
Calculating CI: real line dim = 7 n_band=8 n_sample=500
Calculating AI+: real line dim = 7 n_band=8 n_sample=500
Calculating BDI:S++: real line dim = 7 n_band=8 n_sample=500
Calculating AII: real line dim = 7 n_band=8 n_sample=500
Calculating D: real line dim = 7 n_band=8 n_sample=500


Calculating CII+: real line dim = 7 n_band=8 n_sample=500
Calculating DIII:S--: real line dim = 7 n_band=8 n_sample=500
Calculating AII+: real line dim = 7 n_band=8 n_sample=500



Calculating AIII:

In [ ]:
n_sample = 500
n_band = 8
n_mesh = 1 # very fast mode

def sim_func_Q(Q1, Q2):
    nQ = len(Q1)
    for n in range(nQ):
        _Q = Q1[n]+Q2[n]
        for v in np.linalg.eigvalsh(_Q):
            if np.log10(np.abs(v)) < -10:
                return 0
    return 1

def get_kpoints(n_dim, n_mesh):
    '''
    Get the k points
    '''
    if n_dim == 1:
        kpoints = [[k] for k in np.linspace(0, np.pi, n_mesh)]
    else:
        values = [np.linspace(0, np.pi, n_mesh) for i in range(n_dim)]
        kpoints = list(itertools.product(*values))

    return kpoints

def fast_similarity_func(model1, model2, n_dim, n_mesh=3):
    kpoints = get_kpoints(n_dim, n_mesh=n_mesh)
    Q1 = model1.calculate_Q(kpoints)
    Q2 = model2.calculate_Q(kpoints)
    return sim_func_Q(Q1, Q2)

def worker(parameter, n_band, n_sample, n_mesh):
    non_Hermitian_symmetry_class, n_dim, gap_type = parameter
    print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
        sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))

    generator  = DeformHamiltonianGenerator(
            n=n_band, n_dim=n_dim, 
            non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
            verbose=False)
    hamiltonians = generator.generate(n_sample=n_sample)
    models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
    
    similarity_func = partial(fast_similarity_func, n_dim=n_dim, n_mesh=n_mesh)
    cluster = Cluster(similarity_function=similarity_func)
    center_indices, group_number = cluster.fit(models)
    
    filename = "./data/data_{symmetry_class}_{gap_type}_dim_{n_dim}.h5".format(
        symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim)
    save_data(center_indices, group_number, filename)
    
    #return len(group_number)

#['real line', 'imaginary line', 'point']
n_dims = range(7, 9+1)
parameters = list(itertools.product(non_hermitian_symmetry_list, n_dims, gap_types))
print("Total case: ", len(parameters))

_worker = partial(worker, n_band=n_band, n_sample=n_sample, n_mesh=n_mesh)

n_core = 64
# with Pool(n_core) as pool:
#     pool.map(_worker, parameters)

In [14]:
parameters

[(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)]

In [ ]:
#        group_number_list = np.zeros(len(non_hermitian_symmetry_list))
#         flag = True
#         try:
#             filename = "./data/data_{symmetry_class}_{gap_type}_dim_{n_dim}.h5".format(
#                 symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim)
#         except:
#             flag = False

#         print("Calculating {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample} in parallel with n_core={n_core}".format(
#             gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample, n_core=n_core))

        
#         filename = "./data/data_table_{gap_type}_dim_{n_dim}.h5".format(gap_type=gap_type, n_dim=n_dim)
#         save_data(non_hermitian_symmetry_list, group_number_list, filename)

# Calculating Table (similarity function parallel)

In [ ]:
# n_sample = 100
# gap_index = 0
# gap_type = gap_types[gap_index]

# n_band = 8

# print("Gap type: ", gap_type)
# for n_dim in range(1+1):
#     group_number_list = np.zeros(len(non_hermitian_symmetry_list))

#     for ic, non_Hermitian_symmetry_class in enumerate(non_hermitian_symmetry_list):
#         print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}".format(
#             sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))

#         generator  = DeformHamiltonianGenerator(
#             n=n_band, n_dim=n_dim, 
#             non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
#             verbose=False)

#         hamiltonians = generator.generate(n_sample=n_sample)
#         models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]

#         similarity_func = topology_comparator
#         cluster = Cluster(similarity_function=similarity_func, n_core=2)

#         center_indices, group_number = cluster.fit(models)
#         group_number_list[ic] = len(group_number)

    
#     filename = "./data/data_table_{gap_type}_dim_{n_dim}.h5".format(gap_type=gap_type, n_dim=n_dim)
#     save_data(non_hermitian_symmetry_list, group_number_list, filename)

# Test

In [ ]:
# non_Hermitian_topological_classification_list_0D = {
#     # Complex AZ class
#     # real, imag, point
#     "A":    ['Z', 'Z', '0'], 
#     "AIII": ['0', 'Z+Z', 'Z'],
#     # Real AZ class
#     "AI":   ['Z', 'Z_2', 'Z_2'],
#     "BDI":  ['Z_2', 'Z_2+Z_2', 'Z_2'],
#     "D":    ['Z_2', 'Z_2', '0'],
#     "DIII": ['0', 'Z', '2Z'],
#     "AII":  ['2Z', '0', '0'], 
#     "CII":  ['0', '0', '0'],
#     "C":    ['0', '0', '0'],
#     "CI":   ['0', 'Z', 'Z'],
#     "AI+":  ['Z', 'Z', '0'],
#     "BDI+": ['Z_2', 'Z+Z', 'Z'],
#     # "D+":   ['D', 'AI', 'BDI'],
#     "DIII+": ['0', 'Z', 'Z_2'],
#     "AII+": ['2Z', '2Z', '0'],
#     "CII+": ['0', '2Z+2Z', '2Z'],
#     # "C+":   ['C', 'AII', 'CII'],
#     "CI+":  ['0', 'Z', '0'],
#     # Complex AZ class with sublattice symmetry
#     "A:S":      ['0', '0', '0'], 
#     "AIII:S+":  ['0', '0', '0'],
#     "AIII:S-":  ['Z', 'Z', 'Z+Z'],
#     # Real AZ class with sublattice symmetry
#     "BDI:S++":  ['Z_2+Z_2', 'Z_2+Z_2', 'Z_2'],
#     "DIII:S--":  ['0', '0', '0'],
#     "CII:S++":  ['0', '0', '0'], 
#     "CI:S--":   ['0', '0', '0'],
#     "AI:S-":    ['0', '0', '0'],
#     "BDI:S-+":  ['Z', 'Z_2', 'Z'],
#     "D:S+":     ['Z_2', 'Z_2', '0'],
#     "CII:S-+":  ['2Z', '0', 'Z'],
#     "C:S+":     ['0', '0', '0'],
#     "DIII:S++": ['0', '0', '0'],
#     "CI:S++":   ['0', '0', 'Z_2'],
#     "AI:S+":    ['Z_2', 'Z_2', 'Z_2+Z_2'],
#     "BDI:S+-":  ['Z_2', 'Z_2', 'Z_2+Z_2'],
#     "D:S-":     ['0', '0', '0'],
#     "DIII:S+-": ['2Z', '2Z', '2Z+2Z'],
#     "AII:S+":   ['0', '0', '0'],
#     "CII:S+-":  ['0', '0', '0'],
#     "C:S-":     ['0', '0', '0'],
#     "CI:S+-":   ['Z', 'Z', 'Z+Z'],
# }
# n_sample = 100
# gap_type = gap_types[1]
# n_dim = 0
# non_Hermitian_symmetry_class = 'CII+'
# n_band = 8
# print("Symmetry class {0} with d = {1}".format(non_Hermitian_symmetry_class, n_dim))
# print("The number of samples: ", n_sample)
# print("Gap type: ", gap_type)

# generator  = DeformHamiltonianGenerator(
#     n=n_band, n_dim=n_dim, 
#     non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
#     verbose=True)

# hamiltonians = generator.generate(n_sample=n_sample)
# models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
# similarity_func = topology_comparator
# cluster = Cluster(similarity_function=similarity_func)

# center_indices, group_number = cluster.fit(models)
# plt.bar(range(len(group_number)), group_number)
# plt.xticks(range(len(group_number)), range(1, len(group_number)+1), fontsize=18)
# plt.tight_layout()
# plt.show()